In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.optimizers import Adam, SGD
import splitfolders
import os

c:\Users\konva\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
data_folder = 'results'
splitfolders.ratio(data_folder, output='dataset', ratio=(0.65, 0.2, 0.15), seed=2, group_prefix=None)

Copying files: 14906 files [00:50, 295.99 files/s]


In [2]:
train_size = 0
test_size = 0
val_size = 0

test_path = r'C:\Users\konva\Documents\STUDY\Mashine Learning\7 семестр\7. Нейронная сеть\test'
train_path = r'C:\Users\konva\Documents\STUDY\Mashine Learning\7 семестр\7. Нейронная сеть\train'
val_path = r'C:\Users\konva\Documents\STUDY\Mashine Learning\7 семестр\7. Нейронная сеть\val'

for folder in os.listdir(train_path):
    for file in os.listdir(train_path + '\\' + str(folder)):
        train_size += 1

for folder in os.listdir(test_path):
    for file in os.listdir(test_path + '\\' + str(folder)):
        test_size += 1

for folder in os.listdir(val_path):
    for file in os.listdir(val_path + '\\' + str(folder)):
        val_size += 1
        
print(train_size)
print(test_size)
print(val_size)

9679
2249
2978


In [3]:
train_dir = 'train'                 # Каталог с данными для обучения
val_dir = 'val'                     # Каталог с данными для проверки
test_dir = 'test'                   # Каталог с данными для тестирования
img_width, img_height = 224, 224    # Размеры изображения
# Размерность тензора на основе изображения для входных данных в нейронную сеть backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
batch_size = 64                     # Размер мини-выборки
nb_train_samples = train_size       # Количество изображений для обучения
nb_validation_samples = val_size    # Количество изображений для проверки
nb_test_samples = test_size         # Количество изображений для тестирования

In [4]:
load_net = ResNet152V2(
    include_top=True,
    weights="imagenet",
    input_shape=input_shape
)

In [5]:
load_net.trainable = False

Создаем составную нейронную сеть на основе предобученной

In [6]:
model = Sequential()
model.add(load_net)
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Functional)     (None, 1000)              60380648  
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               200200    
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5), 
              metrics=['accuracy'])

Создаем генератор изображений

In [9]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [10]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 9679 images belonging to 15 classes.


In [11]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2978 images belonging to 15 classes.


In [12]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2249 images belonging to 15 classes.


Обучаем модель с использованием генераторов

In [13]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=6,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/6
151/151 [==============================] - 2374s 16s/step - loss: 0.6759 - accuracy: 0.7748 - val_loss: 0.6680 - val_accuracy: 0.9330
Epoch 2/6
151/151 [==============================] - 2274s 15s/step - loss: 0.6567 - accuracy: 0.9037 - val_loss: 0.6482 - val_accuracy: 0.9333
Epoch 3/6
151/151 [==============================] - 2429s 16s/step - loss: 0.6363 - accuracy: 0.9301 - val_loss: 0.6265 - val_accuracy: 0.9333
Epoch 4/6
151/151 [==============================] - 2360s 16s/step - loss: 0.6138 - accuracy: 0.9332 - val_loss: 0.6022 - val_accuracy: 0.9333
Epoch 5/6
151/151 [==============================] - 2341s 16s/step - loss: 0.5898 - accuracy: 0.9333 - val_loss: 0.5755 - val_accuracy: 0.9333
Epoch 6/6
151/151 [==============================] - 2430s 16s/step - loss: 0.5628 - accuracy: 0.9333 - val_loss: 0.5466 - val_accuracy: 0.9333


Оцениваем качество работы сети с помощью генератора

In [14]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("accuracy на тестовых данных: %.2f%%" % (scores[1]*100))

c:\Users\konva\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1983: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


accuracy на тестовых данных: 93.33%


In [15]:
load_net.trainable = True
trainable = False
for layer in load_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable 

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Functional)     (None, 1000)              60380648  
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               200200    
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-5), 
              metrics=['accuracy'])

c:\Users\konva\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [18]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
151/151 [==============================] - 2254s 15s/step - loss: 0.5472 - accuracy: 0.9333 - val_loss: 0.5459 - val_accuracy: 0.9333
Epoch 2/2
151/151 [==============================] - 2202s 15s/step - loss: 0.5477 - accuracy: 0.9333 - val_loss: 0.5454 - val_accuracy: 0.9333


In [19]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("accuracy на тестовых данных: %.2f%%" % (scores[1]*100))

accuracy на тестовых данных: 93.33%
